# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

План работы:
1. Изучить файл с данными.
2. разбить файл на три выборки: обучающую, валидационную и тестовую.
3. исследовать качество разных моделей, меняя гиперпараметры.
4. проверить качество модели на тестовой выборке.
5. доп. задание: проверить модели на вменяемость.

## Откройте и изучите файл

In [1]:
# импортируем все необходимые библиотеки
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set();# более красивый внешний вид графиков по умолчанию
import numpy as np
from scipy import stats as st

from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

import warnings
warnings.filterwarnings('ignore')

In [2]:
# Считаем данные из csv-файла в датафрейм и сохраним в переменную data. Путь к файлу:/datasets/users_behavior.csv
data = pd.read_csv('/datasets/users_behavior.csv')

In [3]:
data.info()
print('\nЧисло дубликатов:', data.duplicated().sum())
data.describe()
data.columns

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB

Число дубликатов: 0


Index(['calls', 'minutes', 'messages', 'mb_used', 'is_ultra'], dtype='object')

In [4]:
data['is_ultra'].value_counts()

0    2229
1     985
Name: is_ultra, dtype: int64

In [5]:
data.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


**Вывод по шагу 1:**

Предобработка данных проведена на предыдущем этапе. Сейчас мы еще раз в этом убедились:
1. импортировали все необходимые библиотеки,которые будут нужны для дальнейшей работы.
2. изучен файл: в названии столбцов используется нижний регистр, дубликаты и пропуски отсутствуют, несоответствие типов данных не выявлено.

## Разбейте данные на выборки

За целевой признак берем столбец is_ultra. Остальные столбцы (признаки) помогут нам предсказывать решение по целевому признаку. В нашем случае решается задача классификации, так как определенный целевой признак is ultra - категориальный. Когда категорий всего две (0, 1) речь идёт о бинарной (двоичной) классификации.

Разобьем выборку по принципу 60/20/20

In [6]:
train_data, pr_data = train_test_split(data, test_size=0.40, random_state=12345)
valid_data, test_data = train_test_split(pr_data, test_size=0.50, random_state=12345)

print('Размер обучающей выборки', train_data.shape[0])
print('Размер валидационной выборки', valid_data.shape[0])
print('Размер тестовой выборки', test_data.shape[0])

Размер обучающей выборки 1928
Размер валидационной выборки 643
Размер тестовой выборки 643


Составим переменные для признаков и целового признака ('is_ultra') для каждой выборки.

In [7]:
train_features = train_data.drop(['is_ultra'], axis=1)
train_target = train_data['is_ultra']
valid_features = valid_data.drop(['is_ultra'], axis=1)
valid_target = valid_data['is_ultra']
test_features = test_data.drop(['is_ultra'], axis=1)
test_target = test_data['is_ultra']

**Вывод по шагу 2:**

1. Данные были разбиты по принципу 60/20/20.
2. составлены переменный для признаков и целевого признака для каждой выборки.
3. Обучающая модель определена - train_data
4. Валидационная выборка определена - valid_data
5. Тестовая выборка определена - test_data

## Исследуйте модели

### Начнем изучение с модели решающее дерево

In [8]:
best_depth = 0
best_result = 0
for depth in range(1, 20):
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth) # объявляем модель с заданной глубиной дерева
    model.fit(train_features, train_target) # обучение модели
    predictions_valid = model.predict(valid_features) # получение предсказания модели
    result = accuracy_score(valid_target, predictions_valid) # считаем качество модели
    print('Глубина дерева', depth, 'Точность', result)
    if result > best_result:
        best_depth = depth
        best_result = result
        
print('Лучшая глубина', best_depth, 'Лучшая точность', best_result)

Глубина дерева 1 Точность 0.7542768273716952
Глубина дерева 2 Точность 0.7822706065318819
Глубина дерева 3 Точность 0.7853810264385692
Глубина дерева 4 Точность 0.7791601866251944
Глубина дерева 5 Точность 0.7791601866251944
Глубина дерева 6 Точность 0.7838258164852255
Глубина дерева 7 Точность 0.7822706065318819
Глубина дерева 8 Точность 0.7791601866251944
Глубина дерева 9 Точность 0.7822706065318819
Глубина дерева 10 Точность 0.7744945567651633
Глубина дерева 11 Точность 0.7620528771384136
Глубина дерева 12 Точность 0.7620528771384136
Глубина дерева 13 Точность 0.7558320373250389
Глубина дерева 14 Точность 0.7589424572317263
Глубина дерева 15 Точность 0.7465007776049767
Глубина дерева 16 Точность 0.7340590979782271
Глубина дерева 17 Точность 0.7356143079315708
Глубина дерева 18 Точность 0.7309486780715396
Глубина дерева 19 Точность 0.7278382581648523
Лучшая глубина 3 Лучшая точность 0.7853810264385692


При глубине решающего дерева 3 получается лучший результат accuracy 0.7853810264385692

Изменим гиперпараметр 'gini'.

In [9]:
best_depth = 0
best_result = 0
for depth in range(1, 20):
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth, criterion='entropy') # объявляем модель с заданной глубиной дерева
    model.fit(train_features, train_target) # обучение модели
    predictions_valid = model.predict(valid_features) # получение предсказания модели
    result = accuracy_score(valid_target, predictions_valid) # считаем качество модели
    print('Глубина дерева', depth, 'Точность', result)
    if result > best_result:
        best_depth = depth
        best_result = result
        
print('Лучшая глубина', best_depth, 'Лучшая точность', best_result)

Глубина дерева 1 Точность 0.7542768273716952
Глубина дерева 2 Точность 0.7822706065318819
Глубина дерева 3 Точность 0.7853810264385692
Глубина дерева 4 Точность 0.7838258164852255
Глубина дерева 5 Точность 0.7791601866251944
Глубина дерева 6 Точность 0.7853810264385692
Глубина дерева 7 Точность 0.7791601866251944
Глубина дерева 8 Точность 0.7776049766718507
Глубина дерева 9 Точность 0.7822706065318819
Глубина дерева 10 Точность 0.7729393468118196
Глубина дерева 11 Точность 0.7651632970451011
Глубина дерева 12 Точность 0.7651632970451011
Глубина дерева 13 Точность 0.7558320373250389
Глубина дерева 14 Точность 0.7511664074650077
Глубина дерева 15 Точность 0.7511664074650077
Глубина дерева 16 Точность 0.7418351477449455
Глубина дерева 17 Точность 0.7262830482115086
Глубина дерева 18 Точность 0.7262830482115086
Глубина дерева 19 Точность 0.7200622083981337
Лучшая глубина 3 Лучшая точность 0.7853810264385692


Изменим гиперпараметр splitter с  'best' на 'random'

In [10]:
best_depth = 0
best_result = 0
for depth in range(1, 20):
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth, splitter='random') # объявляем модель с заданной глубиной дерева
    model.fit(train_features, train_target) # обучение модели
    predictions_valid = model.predict(valid_features) # получение предсказания модели
    result = accuracy_score(valid_target, predictions_valid) # считаем качество модели
    print('Глубина дерева', depth, 'Точность', result)
    if result > best_result:
        best_depth = depth
        best_result = result
        
print('Лучшая глубина', best_depth, 'Лучшая точность', best_result)

Глубина дерева 1 Точность 0.7200622083981337
Глубина дерева 2 Точность 0.7200622083981337
Глубина дерева 3 Точность 0.7387247278382582
Глубина дерева 4 Точность 0.7527216174183515
Глубина дерева 5 Точность 0.776049766718507
Глубина дерева 6 Точность 0.7682737169517885
Глубина дерева 7 Точность 0.7713841368584758
Глубина дерева 8 Точность 0.7713841368584758
Глубина дерева 9 Точность 0.776049766718507
Глубина дерева 10 Точность 0.7744945567651633
Глубина дерева 11 Точность 0.7822706065318819
Глубина дерева 12 Точность 0.7651632970451011
Глубина дерева 13 Точность 0.7729393468118196
Глубина дерева 14 Точность 0.7698289269051322
Глубина дерева 15 Точность 0.7636080870917574
Глубина дерева 16 Точность 0.7433903576982893
Глубина дерева 17 Точность 0.7402799377916018
Глубина дерева 18 Точность 0.7387247278382582
Глубина дерева 19 Точность 0.7278382581648523
Лучшая глубина 11 Лучшая точность 0.7822706065318819


Гиперпараметр best работает лучше.

### Изучаем модель случайный лес. Изменим в цикле гиперпараметр 'n_estimators'.

In [11]:
best_est = 0
best_result = 0
for est in range(1, 35):
    model = RandomForestClassifier(random_state=12345, n_estimators=est) # объявляем модель с заданным количеством деревьев
    model.fit(train_features, train_target) # обучение модель на тренировочной выборке
    predictions_valid = model.predict(valid_features) # получение предсказания модели
    result = accuracy_score(valid_target, predictions_valid) # посчитаем качество модели на валидационной выборке
    print('Количество деревьев', est, 'Точность', result)
    if result > best_result:
        best_est = est # сохраняем наилучшую модель
        best_result = result #  сохраняем наилучшее значение метрики accuracy на валидационных данных

print('Лучшее количество деревьев', best_est, 'Лучшая точность', best_result)

Количество деревьев 1 Точность 0.7107309486780715
Количество деревьев 2 Точность 0.7636080870917574
Количество деревьев 3 Точность 0.7387247278382582
Количество деревьев 4 Точность 0.7713841368584758
Количество деревьев 5 Точность 0.749611197511664
Количество деревьев 6 Точность 0.7807153965785381
Количество деревьев 7 Точность 0.7682737169517885
Количество деревьев 8 Точность 0.7822706065318819
Количество деревьев 9 Точность 0.7729393468118196
Количество деревьев 10 Точность 0.7853810264385692
Количество деревьев 11 Точность 0.7838258164852255
Количество деревьев 12 Точность 0.7869362363919129
Количество деревьев 13 Точность 0.7822706065318819
Количество деревьев 14 Точность 0.7838258164852255
Количество деревьев 15 Точность 0.7838258164852255
Количество деревьев 16 Точность 0.7869362363919129
Количество деревьев 17 Точность 0.7869362363919129
Количество деревьев 18 Точность 0.7931570762052877
Количество деревьев 19 Точность 0.7884914463452566
Количество деревьев 20 Точность 0.7869362

При количестве деревьев 23 получается лучший результат accuracy 0.7947122861586314.

Изменим гиперпараметр 'gini'.

In [12]:
best_est = 0
best_result = 0
for est in range(1, 60):
    model = RandomForestClassifier(random_state=12345, n_estimators=est, criterion='entropy') # объявляем модель с заданным количеством деревьев
    model.fit(train_features, train_target) # обучение модель на тренировочной выборке
    predictions_valid = model.predict(valid_features) # получение предсказания модели
    result = accuracy_score(valid_target, predictions_valid) # посчитаем качество модели на валидационной выборке
    print('Количество деревьев', est, 'Точность', result)
    if result > best_result:
        best_est = est # сохраняем наилучшую модель
        best_result = result #  сохраняем наилучшее значение метрики accuracy на валидационных данных

print('Лучшее количество деревьев', best_est, 'Лучшая точность', best_result)

Количество деревьев 1 Точность 0.7387247278382582
Количество деревьев 2 Точность 0.7620528771384136
Количество деревьев 3 Точность 0.76049766718507
Количество деревьев 4 Точность 0.7682737169517885
Количество деревьев 5 Точность 0.7620528771384136
Количество деревьев 6 Точность 0.7776049766718507
Количество деревьев 7 Точность 0.7729393468118196
Количество деревьев 8 Точность 0.7822706065318819
Количество деревьев 9 Точность 0.7807153965785381
Количество деревьев 10 Точность 0.7853810264385692
Количество деревьев 11 Точность 0.7807153965785381
Количество деревьев 12 Точность 0.7838258164852255
Количество деревьев 13 Точность 0.7791601866251944
Количество деревьев 14 Точность 0.7822706065318819
Количество деревьев 15 Точность 0.7807153965785381
Количество деревьев 16 Точность 0.7807153965785381
Количество деревьев 17 Точность 0.776049766718507
Количество деревьев 18 Точность 0.7776049766718507
Количество деревьев 19 Точность 0.7776049766718507
Количество деревьев 20 Точность 0.783825816

Лучшая точность с гиперпараметром 'entropy' составляет 0.7993779160186625.

### Изучаем модель логической регрессии

In [13]:
best_iter = 0
best_result = 0
for iter in range(1, 25):
    model = LogisticRegression(random_state=12345, max_iter=iter) # объявляем модель с заданным количеством итераций
    model.fit(train_features, train_target) # обучение модель на тренировочной выборке
    predictions_valid = model.predict(valid_features) # получение предсказания модели
    result = accuracy_score(valid_target, predictions_valid) # посчитаем качество модели на валидационной выборке
    print('Количество итераций', iter, 'Точность', result)
    if result > best_result:
        best_iter = iter # сохраняем наилучшую модель
        best_result = result #  сохраняем наилучшее значение метрики accuracy на валидационных данных

print("Лучшее количество итераций:", best_iter, 'Лучшая точность', best_result)

Количество итераций 1 Точность 0.7060653188180405
Количество итераций 2 Точность 0.7060653188180405
Количество итераций 3 Точность 0.7060653188180405
Количество итераций 4 Точность 0.7060653188180405
Количество итераций 5 Точность 0.7060653188180405
Количество итераций 6 Точность 0.7060653188180405
Количество итераций 7 Точность 0.7060653188180405
Количество итераций 8 Точность 0.7060653188180405
Количество итераций 9 Точность 0.7060653188180405
Количество итераций 10 Точность 0.7060653188180405
Количество итераций 11 Точность 0.7060653188180405
Количество итераций 12 Точность 0.7060653188180405
Количество итераций 13 Точность 0.7060653188180405
Количество итераций 14 Точность 0.7060653188180405
Количество итераций 15 Точность 0.7060653188180405
Количество итераций 16 Точность 0.7060653188180405
Количество итераций 17 Точность 0.7060653188180405
Количество итераций 18 Точность 0.7045101088646968
Количество итераций 19 Точность 0.7076205287713841
Количество итераций 20 Точность 0.707620

**Вывод по шагу 3:**
В ходе расчетов были получены следующие данные:
1. Модель "Решающее дерево" - при глубине 3 точность составляет 0.7853810264385692.
2. При количестве дереьвев 50 точность модели случайного леса составляет 0.7993779160186625.
3. Точность модели логистической регрессии составляет 0.7076205287713841.

Лучший результат показывает модель случайного леса.

## Проверьте модель на тестовой выборке

Оценим точность каждой модели на тестовой выборке.

In [14]:
best_depth = 0
best_result = 0
for depth in range(1, 20):
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth) # объявляем модель с заданной глубиной дерева
    model.fit(train_features, train_target) # обучение модели
    predictions_test = model.predict(test_features) # получение предсказания модели
    result = accuracy_score(test_target, predictions_test) # считаем качество модели
    if result > best_result:
        best_depth = depth
        best_result = result
        
print('Лучшая глубина', best_depth, 'Лучшая точность', best_result)

Лучшая глубина 7 Лучшая точность 0.7993779160186625


In [15]:
best_est = 0
best_result = 0
for est in range(1, 35):
    model = RandomForestClassifier(random_state=12345, n_estimators=est) # объявляем модель с заданным количеством деревьев
    model.fit(train_features, train_target) # обучение модель на тренировочной выборке
    predictions_test = model.predict(test_features) # получение предсказания модели
    result = accuracy_score(test_target, predictions_test) # посчитаем качество модели на валидационной выборке
    if result > best_result:
        best_est = est # сохраняем наилучшую модель
        best_result = result #  сохраняем наилучшее значение метрики accuracy на валидационных данных

print('Лучшее количество деревьев', best_est, 'Лучшая точность', best_result)

Лучшее количество деревьев 34 Лучшая точность 0.7900466562986003


In [16]:
best_iter = 0
best_result = 0
for iter in range(1, 25):
    model = LogisticRegression(random_state=12345, max_iter=iter) # модель с заданным количеством итераций
    model.fit(train_features, train_target) # обучение модель на тренировочной выборке
    predictions_test = model.predict(test_features) # получение предсказания модели
    result = accuracy_score(test_target, predictions_test) # посчитаем качество модели на валидационной выборке
    print('Количество итераций', iter, 'Точность', result)
    if result > best_result:
        best_iter = iter # сохраняем наилучшую модель
        best_result = result #  сохраняем наилучшее значение метрики accuracy на валидационных данных

print("Лучшее количество итераций:", best_iter, 'Лучшая точность', best_result)

Количество итераций 1 Точность 0.6842923794712286
Количество итераций 2 Точность 0.6842923794712286
Количество итераций 3 Точность 0.6842923794712286
Количество итераций 4 Точность 0.6842923794712286
Количество итераций 5 Точность 0.6842923794712286
Количество итераций 6 Точность 0.6842923794712286
Количество итераций 7 Точность 0.6842923794712286
Количество итераций 8 Точность 0.6842923794712286
Количество итераций 9 Точность 0.6842923794712286
Количество итераций 10 Точность 0.6842923794712286
Количество итераций 11 Точность 0.6842923794712286
Количество итераций 12 Точность 0.6842923794712286
Количество итераций 13 Точность 0.6842923794712286
Количество итераций 14 Точность 0.6842923794712286
Количество итераций 15 Точность 0.6842923794712286
Количество итераций 16 Точность 0.6842923794712286
Количество итераций 17 Точность 0.6827371695178849
Количество итераций 18 Точность 0.6827371695178849
Количество итераций 19 Точность 0.6920684292379471
Количество итераций 20 Точность 0.696734

**Вывод по шагу 4:**

Точность модели решающего дерева на тестовой выборке - 0.7993779160186625, точность модели случайного леса на тестовой выборке - 0.7900466562986003, точность модели логистической регрессии на тестовой выборке - 0.6982892690513219.
Можно использовать модель случайного леса, так как точность этой модели практически совпадает с точностью модели решающего дерева, а на валидационной выборке модель случайного леса показала лучшие результаты.

<div style="border:solid blue 2px; padding: 20px">
<font size="3"><b> Комментарий студента 🎓 </b></font>
    
Да, здесь не так получилось, как надо. Я невнимательно прочитала задание.
    Так как из предыдущего шага у нас получился лучший результат у модели случаный лес, поэтому мы возьмем эту модель и проверим корректность работы лучшей модели на тестовой выборке.
</div>

In [17]:
best_est = 0
best_result = 0
for est in range(1, 35):
    model = RandomForestClassifier(random_state=12345, n_estimators=est) # объявляем модель с заданным количеством деревьев
    model.fit(train_features, train_target) # обучение модель на тренировочной выборке
    predictions_test = model.predict(test_features) # получение предсказания модели
    result = accuracy_score(test_target, predictions_test) # посчитаем качество модели на валидационной выборке
    if result > best_result:
        best_est = est # сохраняем наилучшую модель
        best_result = result #  сохраняем наилучшее значение метрики accuracy на валидационных данных

print('Лучшее количество деревьев', best_est, 'Лучшая точность', best_result)

Лучшее количество деревьев 34 Лучшая точность 0.7900466562986003


<div style="border:solid blue 2px; padding: 20px">
<font size="3"><b> Комментарий студента 🎓 </b></font>
    
**Вывод по шагу 4:**
    Точность модели случайный лес на валидационной выборке составляет 0.7947122861586314, а на тестовой - 0.7900466562986003. Модель случайного леса показала лучшие результаты.
</div>

Метрики качества оценивают качество работы и выражаются в числовой форме.
1. accuracy вычисляли выше.
2. precision - точность - можно интерпретировать как долю объектов, названных классификатором положительными и при этом действительно являющимися положительными. Precision = True Positive/ (True Positive + False Positive)
3. recall – это показатель ошибки классификации. Он оценивает результат алгоритмов классификации, для которых значение цели и ответа является категорией. По сути, Recall в Python определяет количество значений, которые правильно предсказаны и на самом деле правильно помечены. Под этим мы подразумеваем, что он представляет собой процент значений, которые были действительно правильно помечены и теперь также правильно предсказаны. Итак, технически говоря, Recall – это показатель ошибки, который учитывает способность классификаторов правильно прогнозировать положительные маркированные образцы. Recall = True Positive/ (True Positive + False Negative)

Метрики качества тесно связаны с исходной задачей классификации

**Оценим модель решающего дерева на адекватность**

In [18]:
tree_model = DecisionTreeClassifier(random_state=12345, max_depth=3)
tree_model.fit(train_features, train_target)
test_predictions = tree_model.predict(test_features)
accuracy = accuracy_score(test_target, test_predictions)
print('Глубина дерева', 3, 'Точность', accuracy)

Глубина дерева 3 Точность 0.7791601866251944


In [19]:
test_predictions = tree_model.predict(test_features)
precision = precision_score(test_target, test_predictions)
recall = recall_score(test_target, test_predictions)
print('precision=', precision, 'recall=', recall)

precision= 0.744 recall= 0.458128078817734


**Оценим модель случайного леса на адекватность**

In [20]:
forest_model = RandomForestClassifier(random_state=12345, n_estimators=23)
forest_model.fit(train_features, train_target)
test_predictions = forest_model.predict(test_features)
accuracy = accuracy_score(test_target, test_predictions)
print('Количество деревьев', 23, 'Точность', accuracy)

Количество деревьев 23 Точность 0.7807153965785381


In [21]:
test_predictions = forest_model.predict(test_features)
precision = precision_score(test_target, test_predictions)
recall = recall_score(test_target, test_predictions)
print('precision=', precision, 'recall=', recall)

precision= 0.6781609195402298 recall= 0.5812807881773399


**Оценим модель логической регрессии на адекватность**

In [22]:
log_model = LogisticRegression(random_state=12345, max_iter=19)
log_model.fit(train_features, train_target)
test_predictions = log_model.predict(test_features)
accuracy = accuracy_score(test_target, test_predictions)
print('Количество итераций', 19, 'Точность', accuracy)

Количество итераций 19 Точность 0.6920684292379471


In [23]:
test_predictions = log_model.predict(test_features)
precision = precision_score(test_target, test_predictions)
recall = recall_score(test_target, test_predictions)
print('precision=', precision, 'recall=', recall)

precision= 0.7272727272727273 recall= 0.03940886699507389


Существует несколько различных способов объединить precision и recall в агрегированный критерий качества. F-мера (в общем случае $\ F_\beta$) — среднее гармоническое precision и recall

F-мера достигает максимума при полноте и точности, равными единице, и близка к нулю, если один из аргументов близок к нулю.

In [24]:
f_score = f1_score(test_target, test_predictions)
print('f=', f_score)

f= 0.07476635514018692


**Вывод по шагу 5:**

По адекватности модели оцениваются по нескольким параметрам: accuracy, precision и recall (это в нашем случае, для модели классификации). Метрики precision и recall складываются в F-меру.

## Итоговый Вывод

Были проделаны следующие действия согласно плана:
1. изучен файл с данными
2. данные разбиты на три выборки: обучающую, валидационную и тестовую.
3. исследованы три модели классификации: Решающее дерево, Случайный лес и Логистическая регрессия.
   Модель "Решающее дерево" - при глубине 3 точность составляет 0.7853810264385692.
   При количестве дереьвев 50 точность модели случайного леса составляет 0.7993779160186625.
   Точность модели логистической регрессии составляет 0.7076205287713841.
   Лучший результат показывает модель случайного леса.
4. модели проверены на тестовой выборке. Точность модели решающего дерева на тестовой выборке - 0.7993779160186625, точность модели случайного леса на тестовой выборке - 0.7900466562986003, точность модели логистической регрессии на тестовой выборке - 0.6982892690513219. Можно использовать модель случайного леса, так как точность этой модели практически совпадает с точностью модели решающего дерева, а на валидационной выборке модель случайного леса показала лучшие результаты. **Точность модели случайный лес на валидационной выборке составляет 0.7947122861586314, а на тестовой - 0.7900466562986003. Модель случайного леса показала лучшие результаты.**
5. оценена адекватность моделей. По адекватности модели оцениваются по нескольким параметрам: accuracy, precision и recall (это в нашем случае, для модели классификации)